# 第四节 互联网金融项目


## 1. 项目背景

* 互联网金融的本质是金融，金融的核心在于风控。对于网贷平台来说，风控能力决定了互联网金融企业的竞争力。逾期客户画像主要在于得出未按时还款客户的整体画像，帮助企业降低坏账率，给放贷与否以及额度给与指导，另外分析各维度的不良贷款率情况，以及放贷比例，可以给企业提出一些有效的参考建议。
* 此次项目数据来自拍拍贷真实业务数据。拍拍贷平台借款端服务包括面向广大个人用户的通用性借款和其他借款。数据是从2015-01-01到2017-01-30的所有信用标的10%的样本。


## 2. 理解数据

In [ ]:
#导入常用的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus']=False #处理中文和坐标负号显示
import seaborn as sns

In [ ]:
import os
os.chdir('E:\云开明培训机构\云开见明培训课件\data summary\第六章')

In [ ]:
data=pd.read_csv('LCIS.csv',dtype ={'ListingId':str})

In [ ]:
data.info()

In [ ]:
data.describe()


* 总共有37个字段，292539 条信贷记录，借款金额区间[100-5000000]，借款期限[1-24]个月，借款利率[7%-24%] ，准入年龄[18-65]岁。

## 3. 数据预处理

####  1.将变量名统一为中文

In [ ]:
columns={'ListingId':'列表序号','recorddate':'记录日期'}

In [ ]:
data.rename(columns=columns,inplace=True)

In [ ]:
data.head()

#### 2.缺失值处理

In [ ]:
data.apply(lambda x: sum(x.isnull())/len(x) ,axis= 0)

In [ ]:
missing_rate = pd.DataFrame(data.apply(lambda x: sum(x.isnull())/len(x) ,axis= 0),columns = ['缺失率'])

In [ ]:
missing_rate.sort_values('缺失率',ascending=False,inplace =True)

In [ ]:
missing_rate[missing_rate['缺失率'] > 0]['缺失率'].apply(lambda x: format(x, '.3%'))#显示成百分比形式

#### 3. 查看缺失原因，记住不能直接删除

* 下次计划还款利息,下次计划还款本金和下次计划还款日期缺失比例最高，且缺失率最高


In [ ]:
data[data['下次计划还款利息'].isnull()].head()

In [ ]:
# 查看标当前状态情况
data[data['下次计划还款利息'].isnull()]['标当前状态'].value_counts()[:10]

* 缺失值较多的三个变量对应的‘标当前状态’大部分为已还清，所以认定为正常值。

In [ ]:
#查看上次还款利息，上次还款本金和上次还款日期 
data[data['上次还款利息'].isnull()]

In [ ]:
data[data['上次还款利息'].isnull()].iloc[:,-9:-1]

*‘上次还款利息’，‘上次还款本金’，‘上次还款日期’属于同一类型，缺失率基本一致。通过查看数据发现分为两种情况：1.未到首次还款日 2，客户逾期上次还款日未进行还款

In [ ]:
data[data['历史成功借款金额'].isnull()]

* 该项数据缺失部分对应的是首次借款成功客户

In [ ]:
# 对记录日期进行分析
data[data['记录日期'].isnull()][['手机认证','户口认证']].head(5)

* 记录日期缺失，其对应的手机认证，户口认证数据出现异常情况，认定是错误值，记录的是婚姻状况和学历状况因此将缺失数据删除

In [ ]:
data.dropna(subset=['记录日期'],how='any',inplace = True)
print('删除缺失后大小',data.shape)

##### 4.重复值处理

In [ ]:
#共106条重复数据
data[data.duplicated()].shape

In [ ]:
data.drop_duplicates(inplace =True)

In [ ]:
print('删除重复值后大小',data.shape)

#### 5. 错误值处理

In [ ]:
# 对手机认证字段进行统计分析
data['手机认证'].value_counts().plot(kind='bar')

In [ ]:
# 户口认证
data['户口认证'].value_counts()

* 户口认证出现了学历等信息
* 手机认证字段出现了婚姻等信息
* 将这些异常或者错误数据剔除

In [ ]:
data = data[(data['手机认证'] =='成功认证') | (data['手机认证'] =='未成功认证')]

In [ ]:
data.shape

In [ ]:
data['户口认证'].value_counts()# 户口认证没有问题

* 请大家按照同样的方法查看一下其他几个认证字段

## 四、构建模型

#### 1、逾期用户画像
逾期用户画像—用户基本信息，年龄、性别、学历分布（该列表借款人学历认证是否成功，成功则表示有大专及以上学历。）

##### 不同性别的放贷比例与逾期表现

In [ ]:
# 计算男性和女性不同的
df_gender = pd.pivot_table(data = data,columns ='标当前状态',index = '性别',values ='列表序号',aggfunc ='size')

In [ ]:
df_gender['借款笔数占比'] = df_gender.apply(np.sum, axis=1) /df_gender.sum().sum()# 计算男女的借款比例

In [ ]:
df_gender['逾期笔数占比']= df_gender['逾期中']/df_gender.sum(axis=1)# 计算男女的逾期占比

In [ ]:
df_gender

In [ ]:
plt.figure(figsize=(16,9))
plt.subplot(121)
plt.bar(x = df_gender.index,height = df_gender['借款笔数占比'],color= ['c','g'])
plt.title('男女借款比例')
plt.subplot(122)
plt.bar(x = df_gender.index,height = df_gender['逾期笔数占比'],color= ['c','g'])
plt.title('男女逾期比例')
plt.suptitle('不同的客户画像情况',size=20)
plt.show()

* 由图可知：
* 拍拍贷的借款用户男性占绝大多数几乎是女性用户的2倍；
* 逾期表现男性用户也高于女性客户，男性用户中逾期人数占比高出女性用户0.75%。

##### *探索不同年龄的放贷比例与逾期表现

In [ ]:
# 按照年龄进行汇总
df_age = data.groupby(['年龄'])['借款金额'].sum()

In [ ]:
df_age = pd.DataFrame(df_age)
df_age['借款金额累计'] = df_age['借款金额'].cumsum()
df_age['借款金额累计占比'] = df_age['借款金额累计']/df_age['借款金额'].sum()

In [ ]:
df_age

In [ ]:
#获取超过80%对应的最小行索引
index_num = df_age[df_age['借款金额累计占比'] > 0.8].index[0]
# 计算累计占比
cum_percent = df_age.loc[index_num,'借款金额累计占比']

In [ ]:
plt.figure(figsize=(16,12))
plt.bar(x = df_age.index,height = df_age['借款金额'], color = 'steelblue', alpha = 0.5, width = 0.7)
plt.axvline(x  = index_num,color='orange',linestyle="--",alpha=0.8)  
df_age['借款金额累计占比'].plot(style='--ob',secondary_y=True)
plt.text(index_num+0.4,cum_percent ,'累计占比为：%.3f%%' % (cum_percent*100), color = 'indianred')
plt.show()

* 年龄段在23-37借款金额占据80%，55岁以上用户放贷金额非常少

* 因此我们进一步将年龄分段为(17, 24] ， (24, 30] ，(30, 36] ，(36, 42] ， (42, 48]， (48, 54] ，(54, 65]，计算不同年龄段的贷款笔数占比和各年龄段的逾期表现。

In [ ]:
data['age_bin'] = pd.cut(data['年龄'],[17,24,30,36,42,48,54,65],right=True)# right代表是否包含在内

In [ ]:
# 计算男不同年龄段的情况
df_age = pd.pivot_table(data = data,columns ='标当前状态',index = 'age_bin',values ='列表序号',aggfunc ='size')

In [ ]:
df_age['借款笔数'] = df_age.apply(np.sum,axis =1)

In [ ]:
df_age

In [ ]:
df_age['借款笔数分布']=df_age['借款笔数']/df_age['借款笔数'].sum()
df_age['逾期占比']=df_age['逾期中']/df_age['借款笔数']
df_age['借款笔数分布%']=df_age['借款笔数分布'].apply(lambda x: format(x, '.3%'))
df_age['逾期占比%']=df_age['逾期占比'].apply(lambda x: format(x, '.3%'))
df_age

In [ ]:
plt.figure(figsize=(12,9))
df_age['借款笔数分布'].plot(kind = 'bar', rot =45,color = 'steelblue', alpha = 0.5)
plt.xlabel('年龄分段情况')
plt.ylabel('借款笔数分布')
df_age['逾期占比'].plot( rot =45,color = 'red', alpha = 1,secondary_y=True)
plt.ylabel('逾期占比情况')
plt.grid(True)
plt.show()

In [ ]:
# # #条形图
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False
plt.bar(x = df_age.index.values, height = df_age['借款笔数分布'].values)
#plt.ylabel('')
plt.title('2017年6个省份的GDP',fontsize = 25)
plt.xlim(0,10)
plt.show()

* 由图可知：年龄段在25-30岁之间的借款笔数最多占44.9%左右，其逾期占比也是最低为3.07%,逾期率最高的年龄段在55以上，逾期率高达5.88%。

##### 不同学历的逾期表现

该数据集中没有客户的具体学历，仅以学历认证是否成功将客户群体分为大专及以上学历、大专以下学历。

In [ ]:
df_edu  =  pd.pivot_table(data = data,columns ='标当前状态',index ='学历认证',values ='列表序号',aggfunc ='size')

In [ ]:
df_edu

In [ ]:
df_edu['借款笔数'] = df_edu.apply(np.sum,axis=1)

In [ ]:
df_edu['借款笔数占比'] = df_edu['借款笔数'] /df_edu['借款笔数'].sum()

In [ ]:
df_edu['逾期占比'] = df_edu['逾期中'] /df_edu['借款笔数']

In [ ]:
df_edu

In [ ]:
plt.figure(figsize=(12,9))
plt.subplot(121)
plt.pie(x = df_edu['借款笔数占比'], # 绘图数据
        labels=['成功认证','未成功认证'], # 添加标签
        colors=['red','yellow'], # 设置饼图的自定义填充色
        autopct='%.1f%%', # 设置百分比的格式，这里保留一位小数
        pctdistance=0.5,  # 设置百分比标签与圆心的距离
        labeldistance = 1.1, # 设置标签与圆心的距离
        startangle = 30, # 设置饼图的初始角度
        radius = 1.2, # 设置饼图的半径
        counterclock = False, # 是否逆时针，这里设置为顺时针方向
        wedgeprops = {'linewidth': 1.5, 'edgecolor':'green'},# 设置饼图内外边界的属性值
        textprops = {'fontsize':20, 'color':'black'}, # 设置文本标签的属性值
        )
plt.title('学历认证占比')
plt.subplot(122)
plt.bar(x = df_edu.index,height = df_edu['逾期占比'],color= ['c','g'])
plt.title('不同学历的人逾期情况')
plt.suptitle('不同学历的人客户画像',size=20)
plt.show()

* 拍拍贷的借款用户大专及以上和大专以下的用户几乎是1:1分布，逾期表现学历较低的用户逾期率高出学历较高用户，1.2%左右。

##### 逾期用户画像—用户行为信息，是否完成淘宝认证，视频认证，户口认证，手机认证，征信认证客户逾期是否表现出差异。


* 结合不同用户行为放贷笔数占比，观察不同用户行为的逾期表现是否存在差异。

* 定义一个函数用来计算不同用户行为下的借款笔数和逾期比率

In [ ]:
def trans(data,col,ind):
    df  =  pd.pivot_table(data = data,columns =col,index =ind,values ='列表序号',aggfunc ='size')
    df['借款笔数'] = df.apply(np.sum,axis=1)
    df['借款笔数占比'] = df['借款笔数'] /df_edu['借款笔数'].sum()
    df['逾期占比'] = df['逾期中'] /df['借款笔数']
    
    plt.figure(figsize=(12,9))
    plt.subplot(121)
    plt.pie(x = df_edu['借款笔数占比'], # 绘图数据
            labels=['成功认证','未成功认证'], # 添加标签
            colors=['red','yellow'], # 设置饼图的自定义填充色
            autopct='%.1f%%', # 设置百分比的格式，这里保留一位小数
            pctdistance=0.5,  # 设置百分比标签与圆心的距离
            labeldistance = 1.1, # 设置标签与圆心的距离
            startangle = 30, # 设置饼图的初始角度
            radius = 1.2, # 设置饼图的半径
            counterclock = False, # 是否逆时针，这里设置为顺时针方向
            wedgeprops = {'linewidth': 1.5, 'edgecolor':'green'},# 设置饼图内外边界的属性值
            textprops = {'fontsize':17, 'color':'black'}, # 设置文本标签的属性值
            )
    plt.title('%s占比' % ind)
    plt.subplot(122)
    plt.bar(x = df_edu.index,height = df_edu['逾期占比'],color= ['c','g'])
    plt.title('不同%s的人逾期情况' % ind)
    plt.suptitle('%s客户画像' % ind,size=20)
    plt.show()
    
    return df

In [ ]:
# 对淘宝认证进行分析
trans(data=data,col = '标当前状态',ind = '淘宝认证')

In [ ]:
#对手机进行认证
trans(data=data,col = '标当前状态',ind = '手机认证')

* 由以上分析可知：

* 淘宝成功认证的用户逾期率明显低于未成功认证的，分析发现拍拍贷平台淘宝认证是区分借款类型是否为电商，其他借款类型无需进行淘宝认证因此我们后续会将淘宝认证纳入借款类型中重新进行分析。

* 手机认证是否成功逾期表现没有明显差异


* 使用以上代码，请同学们依次完成对 视频认证 , 征信认证 和 户口认证 的分析

## 五、结果解读

* 总结：

①在性别差异方面，男性客户借款笔数是女性客户的近两倍，逾期表现男性用户也高于女性客户，男性用户中逾期人数占比高出女性用户0.75%。

②学历方面，拍拍贷的借款用户大专及以上和大专以下的用户几乎是1:1分布，逾期表现学历较低的用户逾期率高出学历较高用户，1.2%左右。

③年龄分布上，23-36岁的用户借款额占平台总借款额的80%是，是平台借款用户的主力军。年龄段在25-30岁之间的借款笔数最多占44.9%左右，其逾期占比也是最低为3.07%，是比较优质的客户群体；逾期率最高的年龄段在55以上，逾期率高达5.88%。

建议

* 逾期用户画像：年龄在36岁以上尤其是55岁上，男性客户，学历在大专及以下。年龄在24-36岁的客户数量占总客户数量的80%，同时逾期率也在平均值以下是比较优质的客户。

* 用户行为方面，视频认证，征信认证和户口认证成功的用户逾期率反而高于为认证成功的用户，建议平台不要过多依赖用户认证，对认证成功的客户也需要严格审核。